# MOSFET Experiments — 2N7000, IRFZ44N, AO3401

**Module 04 — MOSFETs | Notebook 01**

---

Time to get hands-on with real MOSFETs. We'll work with three devices that cover the range of common MOSFET applications:

| Device | Type | Package | Typical Use |
|---|---|---|---|
| **2N7000** | N-channel, enhancement | TO-92 | Logic-level switching, low power |
| **IRFZ44N** | N-channel, enhancement | TO-220 | Power switching, motor drives |
| **AO3401** | P-channel, enhancement | SOT-23 (breakout) | High-side switching |

**Equipment needed:**
- Klein MM300 multimeter
- Fnirsi 2C53T oscilloscope + signal generator
- Bench power supply (0-30V, 0-10A)
- Breadboard + jumper wires
- Components: 2N7000, IRFZ44N, AO3401 (on breakout board), LEDs, assorted resistors

## ESD Warning — Handle MOSFETs with Care!

Before touching any MOSFET:

**The gate oxide is only nanometers thick.** A static discharge that you can't even feel (< 100V) can permanently damage it.

**Precautions:**
1. Touch a grounded metal object before handling MOSFETs
2. Don't slide MOSFETs across surfaces (generates static)
3. Keep MOSFETs in their anti-static packaging until ready to use
4. When breadboarding, insert the MOSFET *after* wiring the rest of the circuit
5. Never leave the gate pin floating in a circuit — always provide a path to a defined voltage

Modern MOSFETs (like the 2N7000) have built-in ESD protection diodes, but these only help against small transients — don't rely on them.

## Concept — Key MOSFET Parameters

### V_GS(th) — Gate Threshold Voltage
The minimum V_GS to begin turning the MOSFET on. But "begin" is the key word — at V_GS(th), only a tiny trickle of current flows. You need V_GS **well above** V_GS(th) for the MOSFET to be fully on.

### R_DS(on) — On-State Resistance
When the MOSFET is fully on, it behaves like a resistor between drain and source. Lower R_DS(on) = less power wasted as heat. This value depends heavily on V_GS — more gate voltage = lower resistance.

### "Logic Level" MOSFETs
A "logic level" MOSFET has a low enough V_GS(th) (typically 1-2V) that it can be driven directly by 3.3V or 5V logic signals. Standard MOSFETs may need 8-10V on the gate to fully turn on.

| Device | V_GS(th) | R_DS(on) | R_DS(on) test V_GS | Logic Level? |
|---|---|---|---|---|
| 2N7000 | 0.8-3.0V | 1.8 Ω (typ) | V_GS = 4.5V | Yes (marginal at 3.3V) |
| IRFZ44N | 2.0-4.0V | 17.5 mΩ | V_GS = 10V | No — needs 10V gate drive |
| AO3401 | -0.5 to -1.5V | 42 mΩ | V_GS = -4.5V | Yes |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Power dissipation comparison: P = I_D^2 * R_DS(on)

I_D = np.linspace(0, 5, 200)  # Drain current in amps

# R_DS(on) values at typical operating gate voltages
devices = {
    '2N7000 (R_DS=1.8Ω @ V_GS=4.5V)': 1.8,
    'IRFZ44N (R_DS=17.5mΩ @ V_GS=10V)': 0.0175,
    'AO3401 (R_DS=42mΩ @ |V_GS|=4.5V)': 0.042,
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: Power dissipation vs current
for name, rds in devices.items():
    P_diss = I_D**2 * rds
    ax1.plot(I_D, P_diss, linewidth=2.5, label=name)

ax1.set_xlabel('Drain Current I_D (A)', fontsize=12)
ax1.set_ylabel('Power Dissipation (W)', fontsize=12)
ax1.set_title('MOSFET Power Dissipation: P = I_D² × R_DS(on)', fontsize=13)
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 5)
ax1.set_ylim(0, 2)

# Right plot: Zoom in on IRFZ44N and AO3401 at higher currents
I_D_high = np.linspace(0, 40, 200)
for name, rds in list(devices.items())[1:]:
    P_diss = I_D_high**2 * rds
    ax2.plot(I_D_high, P_diss, linewidth=2.5, label=name)

ax2.axhline(y=50, color='red', linestyle='--', linewidth=1.5, alpha=0.7, label='IRFZ44N max P_D (with heatsink)')
ax2.set_xlabel('Drain Current I_D (A)', fontsize=12)
ax2.set_ylabel('Power Dissipation (W)', fontsize=12)
ax2.set_title('Power MOSFETs at Higher Currents', fontsize=13)
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 40)
ax2.set_ylim(0, 80)

plt.tight_layout()
plt.show()

print("Key insight: The 2N7000's high R_DS(on) means it wastes significant power at")
print("even moderate currents. It's a small-signal device — don't push more than ~200mA.")
print(f"\nAt 1A: 2N7000 dissipates {1.0**2 * 1.8:.1f}W (way too much!)")
print(f"At 1A: IRFZ44N dissipates {1.0**2 * 0.0175:.4f}W (negligible)")
print(f"At 10A: IRFZ44N dissipates {10.0**2 * 0.0175:.2f}W (very manageable)")

In [ ]:
# Compare power loss: MOSFET switch vs BJT switch

import numpy as np
import matplotlib.pyplot as plt

I_load = np.linspace(0.01, 2.0, 200)  # Load current (A)

# BJT (2N2222): V_CE(sat) ≈ 0.3V at moderate current, rises with current
# P_BJT = V_CE(sat) * I_C + V_BE * I_B
# Plus you need base current: I_B = I_C / beta
V_CE_sat = 0.3 + 0.1 * I_load  # V_CE(sat) rises a bit with current
beta = 100
V_BE = 0.7
I_B = I_load / beta
P_BJT = V_CE_sat * I_load + V_BE * I_B

# MOSFET (2N7000): R_DS(on) = 1.8 Ω at V_GS = 4.5V
R_DS_on_2N7000 = 1.8
P_MOSFET_2N7000 = I_load**2 * R_DS_on_2N7000

# MOSFET (IRFZ44N): R_DS(on) = 17.5 mΩ at V_GS = 10V
R_DS_on_IRFZ44N = 0.0175
P_MOSFET_IRFZ44N = I_load**2 * R_DS_on_IRFZ44N

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(I_load, P_BJT * 1000, 'r-', linewidth=2.5, label='BJT (2N2222) — V_CE(sat) × I_C')
ax.plot(I_load, P_MOSFET_2N7000 * 1000, 'g--', linewidth=2.5, label='2N7000 — I² × 1.8Ω')
ax.plot(I_load, P_MOSFET_IRFZ44N * 1000, 'b-.', linewidth=2.5, label='IRFZ44N — I² × 17.5mΩ')

ax.set_xlabel('Load Current (A)', fontsize=12)
ax.set_ylabel('Switch Power Dissipation (mW)', fontsize=12)
ax.set_title('Power Loss: BJT Switch vs MOSFET Switch', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 2)
ax.set_ylim(0, 1500)

# Find crossover between BJT and 2N7000
# V_CE_sat * I = I^2 * R_DS → I_cross = V_CE_sat / R_DS
I_cross = 0.3 / R_DS_on_2N7000
ax.axvline(x=I_cross, color='gray', linestyle=':', alpha=0.5)
ax.annotate(f'BJT = 2N7000 at ~{I_cross*1000:.0f}mA',
           xy=(I_cross, 200), fontsize=10, color='gray',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"At low currents, the 2N7000 MOSFET wastes LESS power (I²R is tiny).")
print(f"Above ~{I_cross*1000:.0f}mA, the 2N7000's high R_DS(on) makes it worse than a BJT.")
print(f"The IRFZ44N (power MOSFET) beats the BJT at ALL current levels shown.")
print(f"\nBut remember: The IRFZ44N needs 10V gate drive — not logic-compatible.")

In [ ]:
# Gate charge curve concept

import numpy as np
import matplotlib.pyplot as plt

# Simplified gate charge curve for a power MOSFET
# This shows V_GS vs Q_g as you charge the gate with a constant current

# Three phases:
# 1. V_GS rises (charging C_GS)
# 2. Miller plateau — V_GS stays flat while V_DS falls (charging C_GD)
# 3. V_GS rises again to final value

Q_total = 70  # Total gate charge in nC (typical for IRFZ44N)
Q_gs = 15     # Gate-source charge
Q_gd = 30     # Gate-drain (Miller) charge  

V_th = 3.0
V_miller = 4.5  # Miller plateau voltage
V_final = 10.0  # Final gate voltage

# Build piecewise curve
Q = np.array([0, Q_gs, Q_gs + Q_gd, Q_total])
V = np.array([0, V_miller, V_miller, V_final])

# Interpolate for smooth plotting
Q_fine = np.linspace(0, Q_total, 500)
V_fine = np.interp(Q_fine, Q, V)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(Q_fine, V_fine, 'b-', linewidth=2.5)

# Annotate the three phases
ax.axhline(y=V_th, color='red', linestyle=':', alpha=0.5, label=f'V_GS(th) = {V_th}V')
ax.axhline(y=V_miller, color='orange', linestyle='--', alpha=0.5, label=f'Miller plateau = {V_miller}V')

# Phase regions
ax.axvspan(0, Q_gs, alpha=0.1, color='green')
ax.axvspan(Q_gs, Q_gs + Q_gd, alpha=0.1, color='orange')
ax.axvspan(Q_gs + Q_gd, Q_total, alpha=0.1, color='blue')

ax.annotate('Phase 1:\nCharging C_GS\n(V_GS rising)', xy=(Q_gs/2, 2), fontsize=10,
           ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))
ax.annotate('Phase 2: Miller Plateau\nCharging C_GD\n(V_DS is falling!)', 
           xy=(Q_gs + Q_gd/2, 6.5), fontsize=10,
           ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.7))
ax.annotate('Phase 3:\nFinal charge\n(V_GS rising)', xy=(Q_gs + Q_gd + 10, 8), fontsize=10,
           ha='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

# Mark Q_gs and Q_gd
ax.annotate('', xy=(0, -0.5), xytext=(Q_gs, -0.5),
           arrowprops=dict(arrowstyle='<->', color='green', lw=1.5))
ax.text(Q_gs/2, -1.0, f'Q_gs = {Q_gs} nC', fontsize=10, ha='center', color='green')

ax.annotate('', xy=(Q_gs, -0.5), xytext=(Q_gs + Q_gd, -0.5),
           arrowprops=dict(arrowstyle='<->', color='orange', lw=1.5))
ax.text(Q_gs + Q_gd/2, -1.0, f'Q_gd = {Q_gd} nC', fontsize=10, ha='center', color='darkorange')

ax.set_xlabel('Gate Charge Q_g (nC)', fontsize=12)
ax.set_ylabel('V_GS (V)', fontsize=12)
ax.set_title('Gate Charge Curve — The Miller Plateau', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_xlim(-5, Q_total + 5)
ax.set_ylim(-2, 12)

plt.tight_layout()
plt.show()

print("The Miller plateau is where V_DS is collapsing — the MOSFET is actually switching.")
print("During this phase, all the gate driver current goes into charging C_GD,")
print("and V_GS doesn't change. This is the slowest part of the switching transition.")
print(f"\nTotal gate charge Q_g = {Q_total} nC")
print(f"To switch in 100ns, you need I_gate = Q_g / t = {Q_total}nC / 100ns = {Q_total/100*1000:.0f}mA")

## The Material Science Why

### Why R_DS(on) depends on V_GS

The channel resistance depends on how many electrons are in the inversion layer. More V_GS = stronger electric field = more electrons attracted to the surface = thicker, more conductive channel = lower R_DS(on).

This is why datasheets always specify R_DS(on) at a particular V_GS. The IRFZ44N's 17.5 mΩ is measured at V_GS = 10V. At V_GS = 5V, R_DS(on) might be 50 mΩ or more — nearly 3x worse!

### Why R_DS(on) increases with temperature

Higher temperature means more lattice vibrations (phonons) in the silicon. Electrons in the channel scatter more frequently off the vibrating atoms, which reduces their mobility (how easily they move). Less mobility = higher resistance.

This is actually a *feature* for parallel operation: if one MOSFET in a parallel bank gets hotter, its R_DS(on) increases, so it carries less current, and the load redistributes. MOSFETs naturally share current! (BJTs do the opposite — thermal runaway.)

### Why the 2N7000 has high R_DS(on)

The 2N7000 is a small-signal device in a TO-92 package. It has a small die with a thin, narrow channel. The IRFZ44N has a massive die with many parallel MOSFET cells — imagine thousands of tiny MOSFETs all connected in parallel on one chip. More parallel cells = lower total R_DS(on).

## Experiment 1 — Measure V_GS(th) of 2N7000

**Goal:** Find the threshold voltage where the 2N7000 just begins to conduct.

### Circuit

```
    Bench PSU (+5V)
         │
         ├──[1kΩ]──┬── Drain (pin 2)
         │         DMM│   2N7000
         │        (V_DS)  (flat side facing you)
         │          │   pin 1: Source
         │          │   pin 2: Gate  
    Pot  │          │   pin 3: Drain
   0-5V──├──[10kΩ]─┤── Gate (pin 2)
    adj  │    │     │
         │   DMM    │
         │  (V_GS)  │
         │    │     │
         └────┴─────┴── Source (pin 1) ── GND
```

**IMPORTANT:** The 2N7000 pinout (flat side facing you, leads down) is:
- Pin 1 (left): **Source**
- Pin 2 (center): **Gate**  
- Pin 3 (right): **Drain**

### Procedure

1. Wire the circuit with the pot at minimum (0V gate)
2. Connect DMM #1 across Gate-Source to read V_GS
3. Connect DMM #2 across Drain-Source to read V_DS (or measure voltage across the 1kΩ resistor to infer I_D)
4. Slowly increase the pot voltage
5. Note the V_GS where V_DS first starts to drop from 5V — this is approximately V_GS(th)
6. The datasheet says 0.8V to 3.0V — your device should fall in this range

### Expected Results

| V_GS | V_DS | I_D (calc) | State |
|---|---|---|---|
| 0 - 1.5V | ~5.0V | ~0 mA | OFF |
| ~2.0V | ~4.5V | ~0.5 mA | Threshold region |
| ~3.0V | ~2.0V | ~3 mA | Turning on |
| ~4.5V | ~0.2V | ~4.8 mA | Fully on |

**Record your V_GS(th): _____ V**

## Experiment 2 — 2N7000 Switching an LED

**Goal:** Use the 2N7000 as a switch and measure V_DS when on.

### Circuit

```
    +5V (bench PSU)
     │
    [330Ω]  (current limiting for LED)
     │
    [LED]
     │
     ├────── Drain (2N7000 pin 3)
     │
     │       2N7000
     │
     │       Gate (pin 2) ──[10kΩ]── Toggle wire
     │       │                       (connect to +5V = ON,
     │      [10kΩ]                    leave open = OFF via
     │       │                        pull-down resistor)
     ├────── Source (pin 1)
     │
    GND
```

**Note:** The 10kΩ from gate to source is a **pull-down resistor** — it ensures the gate is at 0V (OFF) when not driven. Never leave a MOSFET gate floating!

### Procedure

1. Build the circuit with the toggle wire disconnected (LED should be OFF)
2. Connect the toggle wire to +5V — LED turns ON
3. Measure V_DS across drain-source with DMM:
   - **Expected: V_DS < 0.2V** (the MOSFET is a very good switch!)
4. Calculate I_D: I = (5V - V_LED - V_DS) / 330Ω ≈ (5 - 2.0 - 0.1) / 330 ≈ 8.8 mA
5. Measure V_GS — should be close to 5V (through the 10kΩ with essentially zero gate current)

**Compare with BJT:** If you did this with a 2N3904, V_CE(sat) would be ~0.2V at this current — similar. But the BJT needs base current (0.1-0.5 mA), while the MOSFET gate current is essentially zero.

**Record your V_DS(on): _____ mV**

## Experiment 3 — IRFZ44N: Measuring R_DS(on)

**Goal:** Measure the on-resistance of a power MOSFET at a known current.

### Circuit

```
    Bench PSU (+12V, current limit 1A)
         │
        [12Ω 5W]  (wirewound power resistor → ~1A at 12V)
         │
         ├──────── Drain (IRFZ44N pin 1, tab)
         │
    DMM  │         IRFZ44N  (TO-220 package)
   (V_DS)│         Tab = Drain
         │         Pin 1: Gate
         │         Pin 2: Drain  
    +10V─├─[100Ω]─ Pin 3: Source
   (gate │     Gate
   supply)│    │
         │   [10kΩ]
         │    │
         ├────┴──── Source (pin 3) ── GND
         │
        GND
```

**IRFZ44N pinout (looking at the front, leads down):**
- Pin 1 (left): **Gate**
- Pin 2 (center): **Drain** (also the metal tab!)
- Pin 3 (right): **Source**

### Procedure

1. Set bench supply to 12V with 1A current limit
2. Apply 10V to the gate (use a second supply channel, or a 9V battery + series resistor)
3. Measure V_DS across drain-source with DMM on mV range
4. Measure actual current through the load resistor (use DMM in series, or measure voltage across the known resistor)
5. Calculate R_DS(on) = V_DS / I_D

### Expected Results

At V_GS = 10V, I_D ≈ 1A:
- V_DS should be approximately 17-25 mV
- R_DS(on) = V_DS / I_D ≈ 17-25 mΩ
- Datasheet spec: 17.5 mΩ typical at V_GS = 10V, I_D = 25A

**Record your R_DS(on): _____ mΩ**

## Experiment 4 — AO3401 P-Channel High-Side Switch

**Goal:** Use a P-channel MOSFET for high-side switching.

### Circuit

```
    +5V ──────┬──── Source (AO3401)
              │
              │    AO3401 (P-channel)
              │    (on SOT-23 breakout board)
              │
    Toggle ───┤── Gate
    wire       │    │
   (to GND    │   [10kΩ] pull-up to Source (+5V)
    = ON)     │    │
              │    +5V
              │
              ├──── Drain
              │
             [330Ω]
              │
             [LED]
              │
             GND
```

**How P-channel works (reversed logic):**
- Gate at +5V (same as source): V_GS = 0V → **OFF**
- Gate at GND: V_GS = 0 - 5 = -5V → **ON** (exceeds V_GS(th) of -0.5 to -1.5V)

### Procedure

1. Build circuit with 10kΩ pull-up holding gate at +5V (LED OFF)
2. Touch toggle wire from gate to GND — LED turns ON
3. Measure V_SD (source to drain voltage drop) — should be very small
4. The 10kΩ pull-up ensures the gate defaults to OFF (V_GS = 0)

**Key observation:** The load (LED + resistor) is between drain and ground. The MOSFET connects the load to V+ (high-side). This is the natural configuration for P-channel — no special gate drive voltage needed.

## Experiment 5 — PWM LED Dimming with MOSFET

**Goal:** Use the signal generator to create a PWM signal, drive a MOSFET, and dim an LED. Observe on the oscilloscope.

### Circuit

```
    +5V (bench PSU)
     │
    [330Ω]
     │
    [LED]
     │
     ├──── Drain (2N7000)
     │
     │     2N7000
     │
     │     Gate ──[100Ω]── Signal Generator Output
     │      │               (Square wave, 0-5V, 1kHz)
     │    [10kΩ]
     │      │
     ├───── Source ── GND
     │
    GND

    Oscilloscope:
      CH1 → Gate (signal generator output)
      CH2 → Drain (across LED+resistor)
```

### Signal Generator Settings (Fnirsi 2C53T)

- Waveform: **Square**
- Frequency: **1 kHz**
- Amplitude: **5V peak-to-peak**
- Offset: **+2.5V** (so signal goes from 0V to +5V)
- Duty cycle: start at **50%**

### Procedure

1. Set signal generator to 1 kHz, 50% duty cycle, 0-5V square wave
2. Observe both channels on the scope — you should see:
   - CH1 (gate): clean square wave 0-5V
   - CH2 (drain): inverted — when gate is HIGH, drain is LOW (MOSFET on, pulls drain to GND)
3. **Vary duty cycle:** 10%, 25%, 50%, 75%, 90%
   - Watch LED brightness change — your eyes average the pulses
   - The scope shows the actual on/off switching
4. **Increase frequency:** try 100 Hz, 1 kHz, 10 kHz, 100 kHz
   - At 100 Hz you may see flickering
   - At 1 kHz and above, the LED appears smooth
   - At very high frequencies, watch for the gate waveform to lose its sharp edges (gate capacitance slowing things down)
5. **Measure rise/fall times** on the drain signal using the scope's measurement function

### What to Record

| Duty Cycle | LED Brightness | V_DS(on) | V_DS(off) |
|---|---|---|---|
| 10% | Dim | | |
| 25% | | | |
| 50% | | | |
| 75% | | | |
| 90% | Bright | | |

In [ ]:
# Visualize PWM dimming concept

import numpy as np
import matplotlib.pyplot as plt

t = np.linspace(0, 5, 5000)  # 5 ms
freq = 1000  # 1 kHz

duty_cycles = [10, 25, 50, 75, 90]

fig, axes = plt.subplots(len(duty_cycles), 1, figsize=(12, 10), sharex=True)

for ax, duty in zip(axes, duty_cycles):
    # Generate PWM signal
    period = 1.0 / freq * 1000  # period in ms
    phase = (t % period) / period  # 0 to 1 within each period
    pwm = np.where(phase < duty / 100.0, 5.0, 0.0)  # 0V or 5V
    
    ax.fill_between(t, 0, pwm, alpha=0.3, color='blue')
    ax.plot(t, pwm, 'b-', linewidth=0.5)
    ax.set_ylabel('V_gate (V)', fontsize=10)
    ax.set_ylim(-0.5, 6)
    ax.set_xlim(0, 5)
    
    avg_v = 5.0 * duty / 100.0
    ax.axhline(y=avg_v, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
    ax.text(5.1, avg_v, f'{duty}% duty\navg={avg_v:.1f}V', fontsize=10, 
           va='center', color='red', fontweight='bold')
    ax.grid(True, alpha=0.2)

axes[-1].set_xlabel('Time (ms)', fontsize=12)
axes[0].set_title('PWM LED Dimming — Gate Voltage at Different Duty Cycles (1 kHz)', fontsize=13)

plt.tight_layout()
plt.show()

print("Red dashed line = average voltage seen by the LED.")
print("Your eyes integrate the light — you see brightness proportional to duty cycle.")
print("The MOSFET is either fully ON or fully OFF — minimal power wasted in the switch!")

## Simulation

Try these in Falstad:

- **N-channel MOSFET switch:** [Falstad — MOSFET LED Switch](https://www.falstad.com/circuit/circuitjs.html?ctz=CQAgjCAMB0l3BWcMBMcUHYMGZIA4UA2ATmIxAUgoqoQFMBaMMAKABcQUFCRCUuUPnxDceMAC4AnEJKEiQMWQDMAhgBsAzgEESYKKoWrtAd1NhJ4yJCEHjp2w6fmkwxM7MPnrlx69eAMSEwNwB7AG1WPk50SRAAJRATMBMLUn52VjTsOgAZACUAVwAXPPygA)

- **PWM MOSFET switching:** Try building the PWM circuit in Falstad and observe how the LED (modeled as a diode) responds to different duty cycles.

## Datasheet Connection

### 2N7000 — Key Specs to Verify

| Parameter | Datasheet Value | Your Measurement |
|---|---|---|
| V_GS(th) | 0.8 - 3.0 V | _____ V |
| R_DS(on) at V_GS = 4.5V | 1.8 Ω (typ), 5.0 Ω (max) | _____ Ω |
| V_DS max | 60 V | (don't test!) |
| I_D max | 200 mA | |
| V_GS max | ±20 V | |

### IRFZ44N — Key Specs to Verify  

| Parameter | Datasheet Value | Your Measurement |
|---|---|---|
| V_GS(th) | 2.0 - 4.0 V | _____ V |
| R_DS(on) at V_GS = 10V | 17.5 mΩ (typ) | _____ mΩ |
| V_DS max | 55 V | |
| I_D max (continuous) | 49 A | |
| Q_g (total gate charge) | 63 nC (typ) | |

### AO3401 — Key Specs

| Parameter | Datasheet Value |
|---|---|
| V_GS(th) | -0.5 to -1.5 V |
| R_DS(on) at V_GS = -4.5V | 42 mΩ (typ) |
| V_DS max | -30 V |
| I_D max | -4.0 A |

## Checkpoint Questions

1. **You measured V_GS(th) of your 2N7000.** Did it fall within the datasheet range of 0.8-3.0V? Why is this range so wide?

2. **Why do we use a 10kΩ pull-down resistor on the gate?** What could happen without it?

3. **The IRFZ44N has R_DS(on) = 17.5 mΩ at V_GS = 10V.** If you only apply V_GS = 5V, what happens to R_DS(on)? Why?

4. **In the PWM experiment, the MOSFET switches between fully on and fully off.** Why is this more efficient than using the MOSFET in its linear region to dim the LED?

5. **Why is the P-channel MOSFET natural for high-side switching?** What would you need to do differently to use an N-channel for high-side?

6. **Calculate the power dissipated** in the IRFZ44N when switching 10A with R_DS(on) = 17.5 mΩ. Is a heatsink needed?

7. **You're designing a circuit driven by a 3.3V microcontroller.** Can you use the IRFZ44N directly? Why or why not? What alternatives exist?

---

*Answers:*
1. *Manufacturing variation — the threshold depends on oxide thickness, doping, etc. Each wafer is slightly different. This is normal.*
2. *Ensures gate is at a defined voltage (0V) when not driven. Without it, the gate floats and stray charge could randomly turn the MOSFET on.*
3. *R_DS(on) increases significantly — fewer electrons in the channel. The MOSFET is partially on, wasting power as heat.*
4. *When fully on: P = I²×R_DS(on) (tiny). When fully off: P = 0. In the linear region, both V_DS and I_D are significant → P = V_DS × I_D is high.*
5. *With gate at ground and source at V+, V_GS = -V_supply, which easily turns on the P-channel. An N-channel high-side needs V_GS above V+, requiring a charge pump or bootstrap.*
6. *P = (10)² × 0.0175 = 1.75W. Yes, a heatsink is advisable — the TO-220 alone might get too hot.*
7. *No — IRFZ44N V_GS(th) is 2-4V, and R_DS(on) is specified at 10V. At 3.3V gate drive, it may barely turn on. Use a logic-level MOSFET (like IRLZ44N) or add a gate driver.*